# Multi-Agent Conversation

---

## Pre-requisites

- Create conda env using the following command from the root of this repo: `conda env create -f ./notebooks/multi-agent-systems/pyautogen/conda-env.yaml`.
- Please define the following environment variables in `.env` file:
    - `OPENAI_API_BASE="<ENDPOINT_URL>"`
    - `OPENAI_API_KEY="<API_KEY>"`

## Setup

In [4]:
import warnings
import os
from dotenv import load_dotenv, find_dotenv

warnings.filterwarnings('ignore')
_ = load_dotenv(find_dotenv())

Please find additional info about how to create LLM Configuration [over here](https://microsoft.github.io/autogen/docs/topics/llm_configuration).

In [5]:
llm_config = {
    "model": "gpt-4-turbo",
    "api_type": "azure",
    "base_url": os.environ.get("OPENAI_API_BASE"),
    "api_key": os.environ.get("OPENAI_API_KEY"),
}

## Define an AutoGen agent

In [6]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [7]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Why don't skeletons fight each other?

They don't have the guts.


By default, method `generate_reply` does not remember the previous conversation context. This is because this method does not maintain state between calls.

In [8]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

I'd be happy to share a joke, but I need a little more context. Was there a specific joke you had in mind, or would you like me to come up with a new one for you?


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [9]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [10]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! I'm ready to bring the laughs. You know, they say laughter is the best medicine, which is great because have you seen my health insurance premiums lately? 😂 What’s new with you?

--------------------------------------------------------------------------------
joe (to cathy):

Haha, Cathy, my health insurance premiums are so high, I need a side hustle just to afford the deductible! Speaking of side hustles, I recently taught my dog how to fetch the newspaper. Now, I’m trying to teach him to negotiate for a lower price!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, Joe, that’s a smart move! With those negotiation skills, maybe next you can teach him to bark in "discount" language. Soon enough, he'll be fetching you deals and coupons instead of just the paper. 😄 

## Print some results

You can print out:
- Chat history
- Cost
- Summary of the conversation

In [11]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'name': 'joe',
  'role': 'assistant'},
 {'content': "Hey Joe! I'm ready to bring the laughs. You know, they say "
             'laughter is the best medicine, which is great because have you '
             'seen my health insurance premiums lately? 😂 What’s new with you?',
  'name': 'cathy',
  'role': 'user'},
 {'content': 'Haha, Cathy, my health insurance premiums are so high, I need a '
             'side hustle just to afford the deductible! Speaking of side '
             'hustles, I recently taught my dog how to fetch the newspaper. '
             'Now, I’m trying to teach him to negotiate for a lower price!',
  'name': 'joe',
  'role': 'assistant'},
 {'content': 'Haha, Joe, that’s a smart move! With those negotiation skills, '
             'maybe next you can teach him to bark in "discount" language. '
             "Soon enough, he'll be fetching you deals and coupons instead of "
             "just the paper. 😄 Just

In [12]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-4-turbo-2024-04-09': {'completion_tokens': 167,
                                                                 'cost': 0.00788,
                                                                 'prompt_tokens': 287,
                                                                 'total_tokens': 454},
                                      'total_cost': 0.00788},
 'usage_including_cached_inference': {'gpt-4-turbo-2024-04-09': {'completion_tokens': 167,
                                                                 'cost': 0.00788,
                                                                 'prompt_tokens': 287,
                                                                 'total_tokens': 454},
                                      'total_cost': 0.00788}}


In [13]:
pprint.pprint(chat_result.summary)

('Haha, Joe, that’s a smart move! With those negotiation skills, maybe next '
 'you can teach him to bark in "discount" language. Soon enough, he\'ll be '
 "fetching you deals and coupons instead of just the paper. 😄 Just don't let "
 'him near your credit cards, or he might start shopping for bones online!')


## Get a better summary of the conversation

In [14]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! I'm ready to bring the laughs. You know, they say laughter is the best medicine, which is great because have you seen my health insurance premiums lately? 😂 What’s new with you?

--------------------------------------------------------------------------------
joe (to cathy):

Haha, Cathy, my health insurance premiums are so high, I need a side hustle just to afford the deductible! Speaking of side hustles, I recently taught my dog how to fetch the newspaper. Now, I’m trying to teach him to negotiate for a lower price!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, Joe, that’s a smart move! With those negotiation skills, maybe next you can teach him to bark in "discount" language. Soon enough, he'll be fetching you deals and coupons instead of just the paper. 😄 

> Note: Please note that the cached result was used for the conversation because the input was same. The models was called only for the generation of summary. 

In [15]:
pprint.pprint(chat_result.summary)

('Joe and Cathy are exchanging jokes about high health insurance premiums and '
 'the amusing idea of teaching a dog to fetch newspapers and negotiate for '
 'better deals.')


## Chat Termination

In [16]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [17]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Nice to meet you. I'm always ready with a joke or two up my sleeve. What do you call fake spaghetti? An impasta! 😄 What about you, got any good jokes to share?

--------------------------------------------------------------------------------
joe (to cathy):

Hey Cathy! Nice one, that was quite al-dente! 😄 Here’s one for you: Why don't skeletons fight each other? They don't have the guts! Got any more? Let’s keep the laughter going!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that was spine-tinglingly funny, Joe! Here's another one: Why can't you give Elsa a balloon? Because she will let it go! 🎈 Your turn, keep the funny bone tickles coming!

--------------------------------------------------------------------------------
joe (to cathy):

Oh, Cathy, that one

In [18]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke was about the alligator: "What do you call an alligator in a vest? An investigator!" 😁 Great one, Cathy! Do you have more jokes, or should I take a swing at it?

--------------------------------------------------------------------------------
cathy (to joe):

Actually, I gotta go, Joe! Keep swinging those jokes, and keep the laughter alive. Catch you later! Bye!

--------------------------------------------------------------------------------
